In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.neural_network import MLPRegressor

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g
import src.eval_utils as e_u

c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DungeonMaster3000\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


В файле baseline.ipynb было показано, что использование эмбеддинга для сообщения коммита позволяет увеличить точность моделей. Попробуем tfidf с уменьшением размерности, а также предобученный мультиязыковой кодировщик предложений на основе Transformer и CNN

Загрузим датасет и предобработаем категориальные фичи

In [2]:
df = d_u.get_preprocess_data()

In [3]:
df.sample(3)

,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,Bob,...,Eve,Mallory,Peggy,Trudy,Victor,Wendy,no_work_d,work_d,no_work_h,work_h
135,compose configs moved to traps/docker,4,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
7,Добавил скрипт для запуска контейнера,4,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
280,moved config files to /opt/avssoft/configs,3,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1


In [4]:
X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

Сначала посмотрим результат модели с эмбеддингами сообщений, которые мы получаем с помощью предобученной мультиязыковой модели

In [5]:
msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_idxs, test_idxs = X_train.index, X_test.index

In [7]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [8]:
X_train = np.concatenate((X_train, msg_embs[train_idxs]), axis=1)
X_test = np.concatenate((X_test, msg_embs[test_idxs]), axis=1)

In [9]:
zoo_models = [
    LinearRegression(),
    SGDRegressor(random_state=42),
    Ridge(random_state=42),
    Lasso(random_state=42),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(),
    MLPRegressor(random_state=42, max_iter=100000)
    ]

for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"MSE: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
MSE: 3.9255156937438884

Модель - SGDRegressor(random_state=42)
MSE: 1.5442650283714352

Модель - Ridge(random_state=42)
MSE: 1.3328333886580885

Модель - Lasso(random_state=42)
MSE: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
MSE: 0.86288125

Модель - GradientBoostingRegressor(random_state=42)
MSE: 0.7879865467367375

Модель - SVR()
MSE: 2.155657021426432

Модель - MLPRegressor(max_iter=100000, random_state=42)
MSE: 1.7726122043258286



Сразу заметно улучшение качества при использовании предобученной модели для генерации эмбеддинга предложения по сравнению с tfidf на том же трэйне и тесте(см. baseline.ipynb)

Посмотрим среднюю квадратичную ошибку на кросс-валидации

In [10]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
ss = StandardScaler()
X_eval = ss.fit_transform(X)
X_eval = np.concatenate((X_eval, msg_embs), axis=1)

for model in zoo_models:
    print(f"Модель - {model}")
    mse_mean = -np.mean(cross_val_score(model, X_eval, y, cv=cv, scoring="neg_mean_squared_error"))
    print(f"MSE: {mse_mean}")
    print()

Модель - LinearRegression()
MSE: 3.6631705664438847

Модель - SGDRegressor(random_state=42)
MSE: 1.5791955326983218

Модель - Ridge(random_state=42)
MSE: 1.3792463938249164

Модель - Lasso(random_state=42)
MSE: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
MSE: 1.7560374454365077

Модель - GradientBoostingRegressor(random_state=42)
MSE: 1.470161025026534

Модель - SVR()
MSE: 2.7170945885316122

Модель - MLPRegressor(max_iter=100000, random_state=42)
MSE: 1.4481975235824769



В отличие от фиксированного трэйна и теста на кросс-валидации себя лучше всего показала линейная модель с L2 регуляризацией и однослойная нейронная сеть

Оценим, как понижение размерности эмбеддинга и применение StandardScaler влияет на качество 

Сначала посмотрим на GradientBoostingRegressor

In [11]:
model = GradientBoostingRegressor(random_state=42)

In [12]:
print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print("На оригинальных эмбеддингах + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, scale=True)

На оригинальных эмбеддингах
-1 - 1.470161025026534
На оригинальных эмбеддингах + масштабирование
-1 - 1.470161025026534


In [13]:
print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

С использованием PCA
5 - 1.5173583599353186
10 - 1.424636346846814
15 - 1.486286074085759
20 - 1.5754300114177344
25 - 1.5040941687477412
50 - 1.5894128842628683
100 - 1.7054535460162703
200 - 1.7929972103337974
-1 - 1.470161025026534


In [14]:
print("С использованием PCA + масштабироавние")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca", scale=True)

С использованием PCA + масштабироавние
5 - 1.4887605803400457
10 - 1.4686811725043527
15 - 1.4882309726004117
20 - 1.5340577496920773
25 - 1.4795469969804564
50 - 1.6390088703490686
100 - 1.8527072529366666
200 - 1.818254333906438
-1 - 1.470161025026534


In [15]:
print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

print()

print("С использованием SVD + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

С использованием SVD
5 - 1.5585692467344392
10 - 1.4375897036278857
15 - 1.3885808583025905
20 - 1.4035701591637975
25 - 1.5666620261978133
50 - 1.7105879995366127
100 - 1.8202123338091556
200 - 1.9189262797423408
-1 - 1.470161025026534

С использованием SVD + масштабирование
5 - 1.4938899714482357
10 - 1.5446139933213214
15 - 1.5517560063622462
20 - 1.3736620989400585
25 - 1.5223086813647835
50 - 1.5905919100962342
100 - 1.8703566375404228
200 - 1.9404148404118065
-1 - 1.470161025026534


Для базового градиентного бустинга лучший результат - 1.435900249421395 при понижении размерности эмбеддинга с помощью TruncatedSVD до 15 без применения масштабирования. Однако, если оставлять исходный размер эмбеддинга, то имеем mse - 1.470161025026534.

Теперь протестируем понижение размерности с моделью Ridge

In [16]:
model = Ridge(random_state=42)

print("Ridge:\n")

print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print()

print("На оригинальных эмбеддингах + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, scale=True)

print()

print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

print()

print("С использованием PCA + масштабироавние")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca", scale=True)

print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

print()

print("С использованием SVD + масштабирование")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")

Ridge:

На оригинальных эмбеддингах
-1 - 1.3672444447352596

На оригинальных эмбеддингах + масштабирование
-1 - 1.3791697289464646

С использованием PCA
5 - 1.608595542403173
10 - 1.5053366229869285
15 - 1.4946788705669938
20 - 1.5038457558324692
25 - 1.5183192143949094
50 - 1.4073004243768081
100 - 1.3820707020223455
200 - 1.3690134044912692
-1 - 1.3672444447352596

С использованием PCA + масштабироавние
5 - 1.6132136939606194
10 - 1.5086607226224173
15 - 1.5005420626002066
20 - 1.513415079915537
25 - 1.5253903798315571
50 - 1.411503149902688
100 - 1.3955213867290475
200 - 1.381822417802797
-1 - 1.3791697289464646
С использованием SVD
5 - 1.71560445461086
10 - 1.4789980076190083
15 - 1.4589009634673815
20 - 1.463223667172491
25 - 1.4717254707227023
50 - 1.4050965252565994
100 - 1.3868345545166867
200 - 1.3704707871867614
-1 - 1.3672444447352596

С использованием SVD + масштабирование
5 - 1.7163396876275487
10 - 1.4766688030250386
15 - 1.4608633948782916
20 - 1.4707248116881606
25 - 1.

Лучшая оценка mse - 1.3672444447352596. Она получается на неуменьшенных эмбеддингах и без масштабирования.

Теперь посмотрим на нейронную сеть

In [17]:
# max_iter = 10000, чтобы модель могла сойтись
model = MLPRegressor(random_state=42, max_iter=10000)

In [18]:
print("MLP:\n")

print("На оригинальных эмбеддингах")
e_u.eval_emb_reduction(model, X, y, msg_embs)

print()

print("С использованием PCA")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

print()

print("С использованием SVD")
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="svd")


MLP:

На оригинальных эмбеддингах
-1 - 1.4306585077994984

С использованием PCA
5 - 1.6209707853472843
10 - 1.5899125582683968
15 - 1.7304625558766156
20 - 1.4343316806985988
25 - 1.355574255722322
50 - 1.1795107218432284
100 - 1.2451863619400807
200 - 1.6258557722371747
-1 - 1.4306585077994984

С использованием SVD
5 - 1.7439838509920171
10 - 1.3618563837699025
15 - 1.3834966603108048
20 - 1.3019823870974476
25 - 1.2812788204710277
50 - 1.2165440901652107
100 - 1.2729807970806544
200 - 1.7428456721093604
-1 - 1.4306585077994984


На данный момент нейронная сеть с понижением размерности эмбеддинга с помощью PCA до 50 показала наилучшую оценку mse - 1.2237945874117258

Попробуем немного изменить архитектуру

In [19]:
model = MLPRegressor(hidden_layer_sizes=(300, 150, 25,), random_state=42, max_iter=100000)
e_u.eval_emb_reduction(model, X, y, msg_embs)

-1 - 1.218504096837509


In [20]:
e_u.eval_emb_reduction(model, X, y, msg_embs, reduct="pca")

5 - 1.771290175695747
10 - 1.3437531127799625
15 - 1.3746870281778076
20 - 1.3648431173606363
25 - 1.386822334748024
50 - 1.2269312974082418
100 - 1.2778318338995986
200 - 1.3699219872447168
-1 - 1.218504096837509


При использовании PCA с новой архитектурой мы немного уменьшили mse, но не очень значительно.

Ячейки выше показывает, что при понижении размерности можно достичь примерно такого же качества, что и при эмбеддинге оригинального размера, но проще ничего не трогать и получать аналогичное качество работы модели или даже выше. Также StandardScaler можно не применять, при его использовании средняя квадратичная ошибка на кросс-валидации слегка увеличивается как на бустинге, так и на линейной модели с регуляризацией.

Лучше всего себя показала нейронная сеть с понижением размера эмбеддинга. Также можно не понижать размерность эмбеддинга сообщения коммита, но в таком случае необходимо увеличивать количество слоев. Следующими по качеству предсказания идут Ridge и GradientBoostingRegressor.

Отдельно проверять tfidf в данный момент уже не буду, так как эмбеддинги предложений, полученные с помощью предобученной модели, сразу показали лучший результат.